In [26]:
# Find a GRIB file to download

from datetime import datetime
# https://weather.gc.ca/grib/grib2_glb_25km_e.html
# https://weather.gc.ca/grib/usage_tips_e.html

"""
https://weather.gc.ca/grib/grib2_ens_geps_e.html:

The data can be accessed at the following URLs:
http://dd.weather.gc.ca/ensemble/geps/grib2/raw/HH/hhh/

where:

raw: is a constant string indicating model data is raw (not processed)
HH: model run start, in UTC [00,12]
hhh: forecast hour [000, 003, …, 192, 198, 204, ..., 378, 384] and [000, 003, …, 192, 198, 204, ..., 762, 768] each Thursday at 000UTC

"""

"""
https://weather.gc.ca/grib/grib2_ens_geps_e.html::

The file names have the following nomenclature:
CMC_geps-raw_Variable_LevelType_Level_latlonResolution_YYYYMMDDHH_Phhh_allmbrs.grib2

where:

CMC_geps-raw: constant string indicating that the data is from the Canadian Meteorological Centre (CMC), the model used (geps: Global Ensemble Prediction System) and the data type (raw).
Resolution: Constant string indicating the data resolution (0p5x0p5)
Variable: Variable type included in this file. To consult a complete list, refer to the variables section.
LevelType: Level type. To consult a complete list, refer to the variables section.
Level: Level value. To consult a complete list, refer to the variables section.
YYYYMMDD: Year, month and day of the beginning of the forecast.
HH: UTC run time [00,12]
Phhh: P is a constant character. hhh is the forecast hour [000, 003, …, 192, 198, 204, ..., 378, 384] or [000, 003, …, 192, 198, 204, ..., 762, 768] each Thursday at 000UTC
allmbrs: constant string indicating that all the members of the ensemble are included in this file
grib2: constant string indicating the GRIB2 format is used
Example of file name:
CMC_geps-raw_UGRD_ISBL_0925_latlon0p5x0p5_2010090100_P078_allmbrs.grib2

This file originates from the Canadian Meteorological Center (CMC) and contains the raw data of the Global Ensemble Prediction System (CMC_geps-raw). The data in the file start on September 1st 2010 at 00Z (2010090100). It contains the U-component wind (UGRD) at the isobaric level 925 mb (ISBL_0925) for the forecast hour 78 (P078), for all members (allmbrs) in GRIB2 format (.grib2).
"""

def get_url(forecastHour : str):
    """
    forecastHour : [000, 003, …, 192, 198, 204, ..., 237, 240]
    """
    url = 'http://dd.weather.gc.ca/model_gem_global/25km/grib2/lat_lon/{HH}/{hhh}/'.format(HH='00', hhh=forecastHour)

    # http://dd.weather.gc.ca/model_gem_global/25km/grib2/lat_lon/HH/hhh/
    # http://dd.weather.gc.ca/model_gem_global/25km/grib2/lat_lon/00/000/

    # https://weather.gc.ca/grib/GLB_HR/GLB_latlonp24xp24_P000_deterministic_e.html
    # https://weather.gc.ca/grib/grib2_glb_25km_e.html#variables
    # 4	Temperature	2m above ground	TMP_TGL_2m	Kelvin
    variable = 'TMP'
    levelType = 'TGL'
    level = '2'

    # Projection: projection used for the data. Can take the values [latlon, ps]
    projection = 'latlon.24x.24'

    date = datetime.now().strftime('%Y%m%d00')

    # CMC_glb_Variable_LevelType_Level_projection_YYYYMMDDHH_Phhh.grib2
    filename = 'CMC_glb_{Variable}_{LevelType}_{Level}_{projection}_{YYYYMMDDHH}_P{hhh}.grib2'.format(
        Variable=variable, LevelType=levelType, Level=level, projection=projection, YYYYMMDDHH=date, hhh=forecastHour)

    return '{}{}'.format(url, filename)

print(get_url('000'))


http://dd.weather.gc.ca/model_gem_global/25km/grib2/lat_lon/00/0/CMC_glb_TMP_TGL_2_latlon.24x.24_2020032700_P0.grib2


In [43]:
# 1. Download grib files
import wget
import os

folder = 'grib'
if not os.path.exists(folder):
    os.makedirs(folder)
# The prediction runs for 10 days. 10 days x 24 hours
for forecastHour in ['{:03d}'.format(hour) for hour in range(0, 10*24+1, 3)]:
    gribUrl = get_url(forecastHour=forecastHour)
    print('downloading {}'.format(gribUrl))
    wget.download(gribUrl, out=folder)



In [81]:
import subprocess
# 2. Import grib files into postgress
with psycopg2.connect(host="127.0.0.1", dbname="wps", user="wps", password="wps") as conn:
    # Generate sql
    outfile_name = '{}/output.sql'.format(folder)
    with open(outfile_name, 'w') as outfile:
        process = subprocess.Popen(
            ['raster2pgsql',
            '{folder}/{grib}'.format(folder=folder, grib=filename),
            'wps.temperature'],
            stdout=outfile, 
            stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()
        print(stderr)
    
    for filename in os.listdir(folder):
        if filename.endswith(".grib2"):
            
            if os.path.exists(outfile_name):
                os.remove(outfile_name)
            with open(outfile_name, 'w') as outfile:
                process = subprocess.Popen(
                    ['raster2pgsql',
                     '-F',
                     '{folder}/{grib}'.format(folder=folder, grib=filename),
                     'wps.temperature'],
                    stdout=outfile, 
                    stderr=subprocess.PIPE)
                stdout, stderr = process.communicate()
                print(stderr)
            # Import into postgis
            with conn.cursor() as cursor:
                with open(outfile_name, 'r') as infile:
                    cursor.execute(infile.read())


            #params = '{folder}/{grib} wps.temperature > {folder}/output.sql'.format(folder=folder,grib=filename)
    #         process = subprocess.Popen(
    #             ['raster2pgsql', '{}/{}'.format(folder, filename), 'wps.temperature', '>', '{}/output.sql'.format(folder)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #         #process = subprocess.Popen(['pwd'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #         stdout, stderr = process.communicate()
    #         print(stdout)
    #         print(stderr)
            break


b'Processing 1/1: grib/CMC_glb_TMP_TGL_2_latlon.24x.24_2020032700_P033.grib2\n'


DuplicateTable: relation "temperature" already exists


In [82]:
#3. Import grib files into postgress
with psycopg2.connect(host="127.0.0.1", dbname="wps", user="wps", password="wps") as conn:
    # Import into postgis
    with conn.cursor() as cursor:
        with open(outfile_name, 'r') as infile:
            print('importing {}'.format(outfile_name))
            cursor.execute(infile.read())


ProgramLimitExceeded: out of memory
DETAIL:  Cannot enlarge string buffer containing 0 bytes by 1459960450 more bytes.


In [16]:
# Iterate through the lat/long values for a GRIB file

# from: https://confluence.ecmwf.int/display/ECC/ecCodes+installation

# # install fortran compiler
# brew install gcc
# brew instal netcdf
# brew install openjpeg

# wget https://confluence.ecmwf.int/download/attachments/45757960/eccodes-2.17.0-Source.tar.gz?api=v2
# tar -xzf eccodes-2.17.0-Source.tar.gz
# mkdir build
# cd build
# cmake ../eccodes-2.17.0-Source
# make
# ctest
# make install


# https://pypi.org/project/eccodes/
# https://confluence.ecmwf.int//display/ECC/Releases

# https://confluence.ecmwf.int/display/ECC/Documentation
# http://download.ecmwf.int/test-data/eccodes/html/namespaceec_codes.html

# https://confluence.ecmwf.int/display/ECC/grib_iterator
# how to use an iterator on lat/lon/values for GRIB messages
from eccodes import *
import psycopg2

missingValue = 1e+20  # A value out of range

match_lat = 48.44023
match_lon = -123.38544

kelvin = 273.15

filename = "CMC_glb_TMP_TGL_2_latlon.24x.24_2020032600_P000.grib2"

def example():
    with open(filename, 'rb') as f:
        while 1:
            gid = codes_grib_new_from_file(f)
            if gid is None:
                break

            # Set the value representing the missing value in the field.
            # Choose a missingValue that does not correspond to any real value in the data array
            codes_set(gid, "missingValue", missingValue)

            iterid = codes_grib_iterator_new(gid, 0)
            
            try:
                with psycopg2.connect(host="127.0.0.1", dbname="wps", user="wps", password="wps") as conn:
                    with conn.cursor() as cursor:

                        i = 0
                        while 1:
                            result = codes_grib_iterator_next(iterid)
                            if not result:
                                break

                            [lat, lon, value] = result

                            #if i % 10000 == 0:
#                             tolerance = 0.3
#                             if abs(lat - match_lat) < tolerance and abs(lon - match_lon) < tolerance:

#                             print(lat - match_lat)

                            if value == missingValue:
                                value = 'missing'
                            else:
                                value = value
                                cursor.execute("INSERT INTO temperature (temperature, geom) VALUES (%s, %s)", 
                                              (value, 'POINT({} {})'.format(lat, lon)))

                            print('- {} - lat={} lon={} value={}'.format(i, lat, lon, value))
                                
                            if i > 10:
                                break

                            i += 1
            finally:
                codes_grib_iterator_delete(iterid)
                codes_release(gid)

def main():
    try:
        example()
    except CodesInternalError as err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            sys.stderr.write(err.msg + '\n')
 
        return 1

main()


NotNullViolation: null value in column "temperature_id" violates not-null constraint
DETAIL:  Failing row contains (null, 216.566, 010100000000000000008056C000000000008066C0).


In [10]:
# https://docs.sqlalchemy.org/en/13/_modules/examples/postgis/postgis.html

import psycopg2
with psycopg2.connect(host="127.0.0.1", dbname="wps", user="wps", password="wps") as conn:
    with conn.cursor() as cursor:
        cursor.execute("""CREATE TABLE temperature(
            temperature_id INTEGER PRIMARY KEY,
            temperature float(8),
            geom geometry
        );
        """)

DuplicateTable: relation "temperature" already exists


In [ ]:
# using gdal
# https://trac.osgeo.org/gdal/wiki/BuildingOnMac
# brew install gdal
# raster2pgsql raster_options schema.table_name > output.sql
# raster2pgsql CMC_glb_TMP_TGL_2_latlon.24x.24_2020032600_P000.grib2 wps.temperature > output.sql
#select rid, ST_Value(rast, foo.pt_geom) as v1 from temperature cross join 
#(select ST_SetSRID(ST_Point(-123.38544, 48.44023), 0) as pt_geom) as foo


In [86]:
with psycopg2.connect(host="127.0.0.1", dbname="wps", user="wps", password="wps") as conn:
        with conn.cursor() as cursor:
            cursor.execute('select filename from wps.temperature')
            record = cursor.fetchone()
            print(record)

None


In [88]:
a = 'moo.sql'
a[:-4]

'moo'